 <img src="Images/fc.pos.jpg" class="bg-primary" align = 'left' width=200/>

# Academia de Física 2026
### Exemplo: Caudais do Rio Mondego

Para obter dados podemos começar com a seguinte pergunta ao Chatgpt:
 - *onde posso encontrar dados de caudais de rios em Portugal?*
 
A primeira resposta envia-nos para o SNIRH – Sistema Nacional de Informação de Recursos Hídricos.
O site é [SNIRH-APA](https://snirh.apambiente.pt/).
Etse site não é muito claro, mas clicando em `Dados de Base->Monitorização` e selecionando
 - Redes: hidromética
 - Bacias Hidrográficas: Mondego 
 - `Aplicar filtros`
 
Podemos agora selecionar a estação de `Açude Ponte de Coimbra` e a série de dados `Caudal descarregado médio diário` e ao clicar `Ver/Guardar dados` surge a série  de dados com registo diário de caudais desde 1987 a 2024, que podemos exportar em formato csv. Abre o ficheiro e guarda-o em formato `EXCEL` (.xlsx) e estás pronto a começar a análise.
 

In [ ]:
#Começa sempre por executar esta célula
#%matplotlib inline                                      
import numpy as np                    # modulo numérico , essencial para trabalho cientifico; chamado
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st


In [ ]:
mondego=pd.read_excel("Dados/serie_Mondego.xlsx")
mondego

É mais fácil começar por eliminar no EXCEL as três primeiras linhas e as últimas que são texto de identificação da origem dos dados: Convém tambem escrever o cabeçalho da primeira coluna **DATA**. Depois dessa alteração voltamos a importar os dados:

In [ ]:
mondego=pd.read_excel("Dados/serie_Mondego2.xlsx")
mondego


O que nos interessa é a coluna **descarregado**

In [ ]:
mondego=mondego["descarregado"]
mondego

Podemos começar com uma "vista de olhos" aos dados: o método `describe()` dá informações básicas sobre o número de dados (dias) -- `count`--,  a média dos valores -- `mean` etc.  

In [ ]:
#estatísticas básicas
mondego.describe()

Temos 13314 dias de observação. O máximo caudal foi de 1902 m³/s, mas a média é apenas 68 m³/s  e o percentil 50% apenas 19.5 m³/s. Isto é metade das observações tem valores iguais ou inferiores a 19.5 m³/s. As cheias, o que nos interessa mais,  são raras mas com caudais várias vezes superiores à média. O número de anos de registo é fácil de obter. Podes aceder individualemente a cada uma destas estatíticas por exemplo:

In [ ]:
mondego.count()

<div class="alert alert-block alert-warning">
    <b>Exercício 1</b>
    <p> Calcula o número de anos de observação destes registos.</p>
</div>

In [ ]:
# anos de observação

Um gráfico usando diretamente os métodos gráficos do `Pandas`

In [ ]:
mondego.plot(ms=0,lw=1) #ms é marker size e lw linewidth
plt.grid(True)    

Estamos interessados nos eventos de cheias, que ocorrem raramente, mas em que os caudais são muito superiores ao "normal". Podemos perguntar, por exemplo, qual é o caudal que é ultrapassado apenas uma vez em 100 - probabilidade de $p=0.01$. Os `dataframe`  tem um método para obter esses valores. Mas antes disso vamos investigar esses valores, contando em quantos dias o  caudal é superior a um limite. Isso ilustra a funcionamento de uma *máscara* para extrair valores de um `dataframe`que cumprem um dado critério. Vê o exemplo seguinte:

In [ ]:
x=np.linspace(0,9,10)
mask=x>4
mask

In [ ]:
x[mask]

Como vês, quando o valor de um elemento de  `mask` é `False` o valor respetivo  é retirado do array `x`. 

<div class="alert alert-block alert-warning">
    <b>Exercício 2</b>
    <p> Com base no expemplo acima, obtém a lista de valores do <i>dataframe</i> <b>mondego</b> que ultrapassam 1000 m³ /s e calcula a frequência relativa (estimador da probabilidade).</p>
</div>

In [ ]:
#exercicio 2


<div class="alert alert-block alert-warning">
    <b>Exercício 3</b>
    <p> Com um ciclo,  repete o cálculo acima para uma lista de caudais [1000, 1900] por intervalos de 100.</p>
</div>

In [ ]:
#exercicio 3 

0 valor de 1000 deve estar próximo do percentil:

In [ ]:
1-1.352e-3

No `Pandas` o percentil $q$ é obtido com o método `quantile(q)`. 

In [ ]:
mondego.quantile(0.999)

<div class="alert alert-block alert-warning">
    <b>Exercício 4</b>
    <p> Usando o método da máscara determina a frequência de ocorrência de cheias de caudal superior a 1020.4 m³/s. Verifica se a frequência relativa é próxima 0.001</p>
</div>

In [ ]:
# exercicio 4

Qual é a probabilidade de acontecer uma cheia destas num ano? Um ano tem 365.3 dias. A cheia pode não ocorrer, ocorrer uma vez, duas ,....  A pergunta mais fácil de responder é a seguinte: 
 - Qual é a probabilidade de não acontecer uma destas cheias num ano?
 
Pensa num dado. Qual a probabilidade de não sair 6 num lançamento. Obviamente 5/6. E de não sir um único 6 em dois lançamentos. Há 5 possibilidades de não sair no primeiro e 5 no segundo. Por isso em 36 possibilidades para os dois lançamento há 25 em que não sai um único 6, ou seja

- probabilidade de não haver 6 em dois lançamentos:

$$\frac{25}{36}=\frac{5}{6}\times \frac{5}{6}$$ 

A probabilidade de não acontecer uma cheia de caudal superior a 1024 m³/s , num dia é $q=0.999$; probabilidade de não acontecer  em nenhum de dois dias é $q\times q=q^2$. Em $n$ dias $q^n$. Logo, a probabilidade de ocorrer pelo menos uma cheia  em $n$ dias é $1-q^n$ (estamos a supor eventos independentes, o que é irrealista para caudais diários).

<div class="alert alert-block alert-warning">
    <b>Exercício 5</b>
    <p> Calcula a probabilidade de pelo menos uma cheia de caudal superior a 1020.4 m³ /s ocorrer num ano. Avalia o resultado contando as cheias deste tipo ocorridas no tempo de observação</p>
</div>

In [ ]:
# exercicio 5

Se uma cheia de caudal superior a $C$ ocorre com probabilidade $p$ num ano, em média quando tempo decorre entre tais cheias? O número médio de cheias em $N$ anos é 
$$
\langle n\rangle=pN 
$$
Temos em média uma cheia num número de anos em que  $pN$=1. Ou seja $N_r$, *tempo de recorrência*,  vale $N_r=1/p$.

<div class="alert alert-block alert-warning">
    <b>Exercício 6</b>
    <p> Define uma função de argumento real, $q$ que te dê o tempo de recorrência de cheias de caudal superior ao percentil $1-q$. Calcula a função para $q=10^{3}$ e $q=10^{-4}$. </p>
</div>

In [ ]:
#exercicio 6

Por esta estimativa cheias de valor superior a 1742 m³/s devem ocorrer de 28 em 28 anos. No periodo em análise quantas vezes se verificaram?

In [ ]:
mask=mondego>mondego.quantile(1-1.0e-4)
mondego[mask].count()

### Explorar a distribuição
Podemos tentar  descobrir a distribuição de caudais para valores extremos. Mas como viste o número de observações é muito reduzido e por isso o que se segue é altamente espculativo. Mas em séries de dados mais longas poderia dar algumas indicações úteis. 

Começamos por representar um histograma cumulativo.
Para fazer um hitograma definimos um conjunto de intervalos de caudais (50 neste exemplo) e determinamos a frequência relativa de ocorrência de caudais em cada intervalo.Num histograma cumulativo,  A altura de cada barra é a percentagem de vezes que se observa um valor de caudal inferior ao indicado. 

In [ ]:
hheights,bins,patches =plt.hist(mondego, bins=50, rwidth=.80, cumulative=True, density=True)
plt.grid(True)
#plt.ylim([.99,1])

In [ ]:
# limite inferiores de cada bin
bins

In [ ]:
#ordenada correspondente a cada bin
np.set_printoptions(precision=6)
hheights

In [ ]:
# um grafico em que as abcissas são o ponto médio de cada intervalo (bin)
cd=(bins[1:]+bins[:-1])/2      # o ponto médio de cada bin
plt.plot(cd,hheights,'bs',ms=2)
plt.grid('True')
#plt.ylim([.95,1])

Vamos fazer um ajuste a uma funão com dois parâmtros. Este função foi escolhida por ter uma comportamento parecido com o do gráfico acima. Não te preocupes com a expressão. Basta que reconheças que tem dois parâmetros livres para ajustar, `beta`  e `mu`. Faremos o ajuste aos valores superiores de caudal.

In [ ]:
from scipy.optimize import curve_fit
def model(x,beta,mu):
    maximo=mondego.max()
    return np.tanh(beta*(x-mu)/maximo)

In [ ]:
cd=(bins[1:]+bins[:-1])/2
cds=cd[10:]                       # os 40 caudais mais altos
hh=hheights[10:]                  # frequências relativas aos caudais mais altos
plt.plot(cds,hh,'bs',ms=2)
plt.grid('True'),
plt.ylim([.95,1])

In [ ]:
parms,cov=curve_fit(model,cds,hh)
parms,cov

In [ ]:
# gráfico conjunto
plt.plot(cds,hh,'bs', cds, model(cds,parms[0],parms[1]),'r-',ms=2)
plt.grid('True'),
plt.ylim([.95,1])

In [ ]:
# o ajuste completo
cd=(bins[1:]+bins[:-1])/2
plt.plot(cd,hheights,'bs', cd, model(cd,parms[0],parms[1]),'r-',ms=2)
plt.grid('True')
#plt.ylim([.95,1])

Usando o modelo podemos estimar a probabilidade de ter uma caudal de 2000 m³ /s o valor crítico que tanto assustou a proteção civil. Na série de dados obtida esse valor nunca foi observado.

In [ ]:
pext = model(2000,parms[0],parms[1])
pext

<div class="alert alert-block alert-warning">
    <b>Exercício 7</b>
    <p> Calcula o tempo de recorrência para o percentil pext </p>
</div>

In [ ]:
# faz aqui o teu exercício

O tempo de recorrência para este valor extremo é de 102 anos e no período de observação, foi observado uma vez. AVISO: estamos a lidar com eventos muito raros e as estatísticas são pouco fiáveis. 

In [ ]:
msk=mondego>mondego.quantile(pext)
mondego[msk].count()